# WIP: make this way more detailed 

In [1]:
import os
import openai

In [2]:
os.environ["OPENAI_API_KEY"] = "sk-..."
openai.api_key = os.environ["OPENAI_API_KEY"]

In [2]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    load_index_from_storage,
    StorageContext,
)
from IPython.display import Markdown, display

INFO:numexpr.utils:Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
NumExpr defaulting to 8 threads.


In [3]:
# load documents
documents = SimpleDirectoryReader(
    "../../../examples/paul_graham_essay/data"
).load_data()

In [4]:
index = VectorStoreIndex.from_documents(documents)

In [5]:
# set Logging to DEBUG for more detailed outputs
query_engine = index.as_query_engine(response_mode="tree_summarize")

In [6]:
prompts_dict = query_engine.get_prompts()

In [7]:
display({k: p.get_template() for k, p in prompts_dict.items()})

{'summary_template': 'Context information from multiple sources is below.\n---------------------\n{context_str}\n---------------------\nGiven the information from multiple sources and not prior knowledge, answer the query.\nQuery: {query_str}\nAnswer: '}

In [8]:
response = query_engine.query("What did the author do growing up?")
print(str(response))

The author worked on writing and programming outside of school before college. They wrote short stories and tried writing programs on an IBM 1401 computer using an early version of Fortran. They also mentioned getting a microcomputer, specifically a TRS-80, and started programming on it.


In [9]:
# customize the prompt

In [10]:
from llama_index.prompts import PromptTemplate

# shakespeare! 
new_summary_tmpl_str = (
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the query in the style of a Shakespeare play.\n"
    "Query: {query_str}\n"
    "Answer: "
)
new_summary_tmpl = PromptTemplate(new_summary_tmpl_str)

In [ ]:
query_engine.update_prompts(summary_template=new_summary_tmpl)

In [12]:
prompts_dict = query_engine.get_prompts()

In [13]:
display({k: p.get_template() for k, p in prompts_dict.items()})

{'summary_template': 'Context information is below.\n---------------------\n{context_str}\n---------------------\nGiven the context information and not prior knowledge, answer the query in the style of a Shakespeare play.\nQuery: {query_str}\nAnswer: '}

In [14]:
response = query_engine.query("What did the author do growing up?")
print(str(response))

In his youth, ere college days did commence,
The author toiled on two pursuits intense.
The first, a writer he did aspire to be,
Crafting tales of characters, though not with glee.
Short stories, lacking plot, did he create,
Thinking them deep, with feelings strong, innate.

But programming, too, did captivate his mind,
With IBM 1401, a machine he did find.
In the basement of his school, a lair so grand,
With alien machines, a sight so grand.
Fortran, the language, he did employ,
Punch cards and printers, his tools of joy.

Yet puzzled was he, with the 1401's might,
No data stored, no input in sight.
Calculating pi, a task he could not pursue,
For math knowledge lacked, his options few.
No programs of note, he could recall,
Save for one that didn't terminate at all.

Microcomputers, a revolution profound,
Changed the game, a new world to be found.
A friend built one, a kit from Heathkit's store,
Envy and awe, the author's heart did soar.
Years of nagging, his father did relent,
A TRS-8